In [ ]:
import cv2
import numpy as np
import os
import json
import matplotlib.pyplot as plt
import matplotlib
from PIL import Image

import data.data_iterators as data

In [ ]:
# Load datasets
IMAGES_DIR = '/media/pupa/DataStorage/datasets'
ANNOTATIONS_DIR = '/home/pupa/dev/apple_face/annotations'

_300w_data = list(data.get_300W_all(IMAGES_DIR, ANNOTATIONS_DIR))
celeba_data = list(data.get_celeba_all(IMAGES_DIR, ANNOTATIONS_DIR))
ffhq_data = list(data.get_ffhq(IMAGES_DIR, ANNOTATIONS_DIR))
wflw_data = list(data.get_WFLW(IMAGES_DIR, ANNOTATIONS_DIR))
all_data = [*_300w_data, *celeba_data, *ffhq_data, *wflw_data]

print(f'300W: {len(_300w_data)}, faces: {sum(len(d.faces) for d in _300w_data)}')
print(f'CelebA: {len(celeba_data)}, faces: {sum(len(d.faces) for d in celeba_data)}')
print(f'FFHQ: {len(ffhq_data)}, faces: {sum(len(d.faces) for d in ffhq_data)}')
print(f'WFLW: {len(wflw_data)}, faces: {sum(len(d.faces) for d in wflw_data)}')
print(f'All: {len(all_data)}, faces: {sum(len(d.faces) for d in all_data)}')

In [ ]:
def draw_landmarks_crop(img, landmarks):
    for landmark in landmarks:
        x, y = int(landmark[0]), int(landmark[1])
        img = cv2.circle(img, (x, y), 3, (0, 0, 255), -1)
    return img

def draw_bboxes(img, bbox):
    left, top, right, bottom = bbox['left'], bbox['top'], bbox['right'], bbox['bottom']
    img = cv2.rectangle(img, (left, top), (right, bottom), (255, 0, 0), 1)
    img = cv2.circle(img, (left, top), 7, (0, 0, 0), -1)
    img = cv2.circle(img, (right, bottom), 7, (255, 255, 255), -1)
    return img

samples_data: list[data.ImageWithFaces] = [
    _300w_data[0],
    celeba_data[0],
    ffhq_data[0],
    wflw_data[0],
]
samples_images = []
for s in samples_data:
    img = cv2.cvtColor(cv2.imread(s.image_path), cv2.COLOR_BGR2RGB)
    print(s.image_path)
    for face in s.faces:
        bbox, landmarks = face.bbox, face.landmarks
        print(bbox)
        img = draw_bboxes(img, bbox)
        img = draw_landmarks_crop(img, landmarks)
    samples_images.append(img)
min_height = min([img.shape[0] for img in samples_images])
samples_images = [cv2.resize(img, (int(img.shape[1] * min_height / img.shape[0]), min_height)) for img in samples_images]
samples_images = np.concatenate(samples_images, axis=1)
plt.figure(figsize=(20, 20))
plt.imshow(samples_images)

In [ ]:
images_count = 0
face_per_img = {}
image_sizes = []
face_sizes = []
for item in all_data:
    images_count += 1
    image_sizes.append((item.image_size[0], item.image_size[1]))
    face_per_img[len(item.faces)] = face_per_img.get(len(item.faces), 0) + 1
    for face in item.faces:
        bbox = face.bbox
        width = bbox['right'] - bbox['left']
        height = bbox['bottom'] - bbox['top']
        assert width > 0 and height > 0, item.image_path
        face_sizes.append((width, height))

In [ ]:
mylist = [key for key, val in face_per_img.items() for _ in range(val)]
plt.hist(mylist, bins=64, log=True, )
plt.title('Face count per image')
plt.show()

In [ ]:
image_sizes = np.array(image_sizes)
plt.hist2d(image_sizes[:, 0], image_sizes[:, 1], bins=64, norm=matplotlib.colors.LogNorm())
plt.xlabel('width')
plt.ylabel('height')
plt.title('Image sizes')
plt.show()

In [ ]:
face_sizes = np.array(face_sizes)
plt.hist2d(face_sizes[:, 0], face_sizes[:, 1], bins=48, norm=matplotlib.colors.LogNorm())
plt.xlabel('width')
plt.ylabel('height')
plt.title('Face sizes')
plt.show()